# Imports

In [26]:
import numpy as np
import luch_basics as lb
import luch_elements as le
import luch_graphics as lg
from bokeh.plotting import figure, output_file, show
from bokeh.models import PrintfTickFormatter, HoverTool, Range1d, LabelSet, Label, WheelZoomTool
from bokeh.resources import CDN
from bokeh.embed import autoload_static
from bokeh.io import output_notebook,show, output_notebook, push_notebook, export_png
output_notebook()


Loading BokehJS ...

# Define beamline

In [27]:
#--------------------- particle start definition---------------------------------
start1 = np.array([[0.000], [0.03], [0.000], [0.03], [0], [0]]) #create start position for 1-st particle
start2 = np.array([[0.003], [0.00], [0.003], [0.00], [0], [0]]) #create start position for 2-nd particle


# ------------------------- beamline definition---------------------------

drift_length_1 = 0.9 # meters
                                   
drift_length_2 = 1.2 # meters
                                   
drift_length_4 =0.4 # meters
                                  
# drift after last optical element
drift_length_3 = 4.3 # meters

quad_voltage = 5075 #volts
quad_atigmatism = -9.6 #percent

dr = -0.3 # mm

tube_lens_voltage = -55.5 # kV
                                      
terminal_voltage = -460 # kV

gap = 0.025 #gap between electrodes in the accelerator tube

gap1 = le.driftspace(length=drift_length_1)
gap2 = le.driftspace(length=drift_length_2)
gap3 = le.driftspace(length=drift_length_3)
gap4 = le.driftspace(length=drift_length_4)

#create dipole magnet
dipole1 = le.dipole(radius=0.33, angle=90, gap=0.055, index=0, pfa1=30.615, pfa2=30.615, K=0.6)

# create variation of electrode opening

Radius = np.ones(40) * 0.030 #create a numpy array of equal 30 mm openings
Radius = np.array([Radius[i] + i * dr / 1000  for i in range(len(Radius))]) # linearly modulates radius

#create accelerator tube
Voltages_tube = np.linspace(0, terminal_voltage, 38) # linearly sets tube voltages
Voltages_lens = np.array([0, tube_lens_voltage]) # sets tube lens voltage
Voltages = np.concatenate((Voltages_lens, Voltages_tube), axis=None) # assembles lens with a tube
acc_tube = le.tube(r=Radius, V=Voltages, gap=gap) # generate accelerator tube

# build quadrupole triplet
quad1 = le.quad(r=0.025, V=quad_voltage * (1 + quad_atigmatism / 100), L=0.102, dir=1)
quad2 = le.quad(r=0.025, V=quad_voltage, L=0.175, dir=0)
quad3 = le.quad(r=0.025, V=quad_voltage * (1 + quad_atigmatism / 100), L=0.102, dir=1)

# -------------------------beamline assembly-------------------
  
beamline = lb.BeamLine([gap1, dipole1, gap2, acc_tube, gap4, quad1, quad2, quad3, gap3])

# Particle tracing

In [28]:
proton1 = lb.Particle(1, 30, 1, start1) #create first particle
proton2 = lb.Particle(1, 30, 1, start2) #create second particle

ray1 = beamline.trace(proton1)
ray2 = beamline.trace(proton2)

plot_x = np.array(ray1.trajectory[:, 6]).reshape(-1, )
plot_y = np.array(ray1.trajectory[:, 0]).reshape(-1, )
plot_y1 = -1 * np.array(ray1.trajectory[:, 2]).reshape(-1, )
plot_y2 = np.array(ray2.trajectory[:, 0]).reshape(-1, )
plot_y3 = -1 * np.array(ray2.trajectory[:, 2]).reshape(-1, )

# ------------------------ graphics ---------------------

plot = lg.default_graph()

# generate the figure
plot.line(plot_x, plot_y, line_width=2, muted_alpha=0.2, color='blue',
          legend_label='divergent')  # add curve as a line to the figure
plot.line(plot_x, plot_y1, line_width=2, muted_alpha=0.2, color='blue',
          legend_label='divergent')  # add curve as a line to the figure
plot.line(plot_x, plot_y2, line_width=2, muted_alpha=0.2, color='red',
          legend_label='parallel')  # add curve as a line to the figure
plot.line(plot_x, plot_y3, line_width=2, muted_alpha=0.2, color='red',
          legend_label='parallel')  # add curve as a line to the figure

show(plot)